In [82]:

%matplotlib qt
%gui qt

import re
import os

import numpy as np
from loguru import logger

import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import pyqtgraph as pg
import pyqtgraph.opengl as gl

from collections import defaultdict
from datetime import datetime

import torch
import gpytorch
from sklearn.preprocessing import StandardScaler


In [83]:
def gatherCSV(rootPath, outsuffix = 'Tracking'):
    '''==================================================
        Collect all EIS.csv files in the rootPath
        Parameter: 
            rootPath: current search path
            outsuffix: Saving path of EIS.csv files
        Returen:
            EISDict: a 2D-dict of EIS data
            Storage Frame: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        ==================================================
    '''
    _filename       = None
    _filepath       = None
    _trackpath      = None
    _csvpath        = None
    _sessionIndex   = None
    _channelIndex   = None
    _processed      = None

    EISDict = defaultdict(dict)

    ## Iterate session
    session_pattern = re.compile(r"(.+?)_(\d{8})_01")
    bank_pattern    = re.compile(r"([1-4])")
    file_pattern    = re.compile(r"EIS_ch(\d{3})\.csv")

    ## RootDir
    for i in os.listdir(rootPath):
        match_session = session_pattern.match(i)
        ## SessionDir
        if match_session:
            logger.info(f"Session Begin: {i}")
            _sessionIndex = match_session[2]
            for j in os.listdir(f"{rootPath}/{i}"):
                match_bank = bank_pattern.match(j)
                ## BankDir
                if match_bank:
                    logger.info(f"Bank Begin: {j}")
                    _trackpath = f"{rootPath}/{i}/{j}/{outsuffix}"
                    if not os.path.exists(_trackpath):
                        continue

                    for k in os.listdir(f"{rootPath}/{i}/{j}/{outsuffix}"):
                        match_file = file_pattern.match(k)
                        ## File
                        if match_file:
                            _filename = k
                            _filepath = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            _channelIndex = (int(match_bank[1])-1)*32+int(match_file[1])
                            
                            EISDict[_sessionIndex][_channelIndex] = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            
    return EISDict

In [84]:
rootPath = "D:/Baihm/EISNN/Dataset/01037160_归档"
# rootPath = "D:/Baihm/EISNN/Archive/01067094_归档"
EISDict = gatherCSV(rootPath)

2025-02-26 19:01:26.143 | INFO     | __main__:gatherCSV:32 - Session Begin: 01037160_20241124_01
2025-02-26 19:01:26.144 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-02-26 19:01:26.145 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-02-26 19:01:26.145 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-02-26 19:01:26.146 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-02-26 19:01:26.147 | INFO     | __main__:gatherCSV:32 - Session Begin: 01037160_20241125_01
2025-02-26 19:01:26.147 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-02-26 19:01:26.148 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-02-26 19:01:26.149 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-02-26 19:01:26.149 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-02-26 19:01:26.150 | INFO     | __main__:gatherCSV:32 - Session Begin: 01037160_20241126_01
2025-02-26 19:01:26.150 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-02-26 19:01:26.151 | INFO    

In [85]:
# Data Readout
def readChannel(chID, fileDict):
    '''==================================================
        Read EIS.csv file by Channel
        Parameter: 
            chID: channel index
            fileDict: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        Returen:
            freq: frequency
            Zreal: real part of impedance
            Zimag: imaginary part of impedance
        ==================================================
    '''
    chData = []
    for ssID in EISDict.keys():
        _data   = np.loadtxt(fileDict[ssID][chID], delimiter=',')
        _freq   = _data[:,0]
        _Zreal  = _data[:,1] * np.cos(np.deg2rad(_data[:,2])) 
        _Zimag  = _data[:,1] * np.sin(np.deg2rad(_data[:,2])) 
        chData.append(np.stack((_freq, _Zreal, _Zimag),axis=0))

    return np.stack(chData, axis=0)


In [93]:

# 这里每个channel都是一个独立样本，这里先举一个例子，以ch069为例
channelIndex = 20
chData = readChannel(channelIndex, EISDict)

# 是否需要Normalization
NormFlag = True

# 根据EISDict的key确定日期范围，然后把日期范围映射到0~days
# Speed Rate = 10 means 1 day = 10 points
SPEED_RATE = 1
x_day = [datetime.strptime(date, '%Y%m%d') for date in EISDict.keys()]
x_train = np.array([(poi - x_day[0]).days for poi in x_day])
x_eval = np.linspace(0,max(x_train),max(x_train)*SPEED_RATE+1)

n_day = np.max(x_train)

logger.info(f"\nx: {np.shape(x_train)} \ny: {np.shape(chData)} \nx_pred{np.shape(x_eval)}")


2025-02-26 19:06:19.248 | INFO     | __main__:<module>:17 - 
x: (16,) 
y: (16, 3, 5000) 
x_pred(17,)


In [ ]:
# Single Point  Gaussian Process Regression
# 这个脚本中，我们把不同频率视为相互独立的变量进行考察
# 由于阻抗仍是一个复数，所以这里使用实部虚部两个task进行GP

class ComplexGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=2
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.RBFKernel(),num_tasks=2, rank=1
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


def ComplexGPTrain(x_train, y_train, x_eval, device, training_iter = 50):
    # Data Normalize
    

    # Initialize likelihood and model
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=2).to(device)
    model = ComplexGPModel(x_train, y_train, likelihood).to(device)

    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # logger.info(f"Training for {training_iter} iterations...")
    for i in range(training_iter):
        optimizer.zero_grad()
        output = model(x_train)
        loss = -mll(output, y_train)
        loss.backward()
        # logger.info(f"Iter {i+1}/{training_iter} - Loss: {loss.item()}")
        optimizer.step()

    # Get into evaluation (predictive posterior) mode
    # logger.info("Model Training Finished.")
    model.eval()
    likelihood.eval()
    # logger.info("Model Evaluation Begin.")
    # Make predictions
    # with torch.no_grad(), gpytorch.settings.fast_pred_var():
    with torch.no_grad(), gpytorch.settings.cholesky_jitter(1e-4):
        observed_pred = likelihood(model(x_train))
        pred = likelihood(model(x_eval))
    # logger.info("Model Evaluation Finished.")

    return observed_pred, pred


In [95]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   
x_train_tensor = torch.tensor(x_train).to(device).float()
x_eval_tensor = torch.tensor(x_eval).to(device).float()


freq_list = np.linspace(0,np.shape(chData)[2]-1,101,dtype=int)

EIS_GP  = []
EIS_org = []
n_RI = np.shape(x_eval)[0]  

# for i in range(np.shape(chData)[2]):
for i in freq_list:
# for i in range(449,450):
    # Training Data Normalizzation
    if NormFlag:
        scaler_real = StandardScaler()
        scaler_imag = StandardScaler()

        y_train_real = scaler_real.fit_transform(chData[:,1,i].reshape(-1,1))
        y_train_imag = scaler_imag.fit_transform(chData[:,2,i].reshape(-1,1))
    else:
        y_train_real = chData[:,1,i].reshape(-1,1)
        y_train_imag = chData[:,2,i].reshape(-1,1)

    y_train_tensor = torch.tensor(np.hstack([y_train_real, y_train_imag]) , device=device).float()

    if torch.isnan(y_train_tensor).any():
        logger.info(f"Freq: {i} - NaN Detected")
        _poi_zero = np.zeros((n_RI))
        EIS_GP.append([[_poi_zero,_poi_zero],[_poi_zero, _poi_zero]])
        continue

    observed_pred, pred = ComplexGPTrain(x_train_tensor, y_train_tensor, x_eval_tensor, device, training_iter = 50)
    mean_pred_norm = pred.mean.cpu().numpy()
    var_pred_norm = pred.covariance_matrix.cpu().numpy()

    if NormFlag:
        mean_pred = np.hstack(
            [scaler_real.inverse_transform(mean_pred_norm[:,0].reshape(-1, 1)),
            scaler_imag.inverse_transform(mean_pred_norm[:,1].reshape(-1, 1))]
        )

        var_scale = mean_pred.transpose().reshape(-1,1) @ mean_pred.transpose().reshape(1,-1)
        var_pred = var_scale * var_pred_norm
    else:
        mean_pred = np.hstack(
            [mean_pred_norm[:,0].reshape(-1, 1),
            mean_pred_norm[:,1].reshape(-1, 1)]
        )
        var_pred = var_pred_norm

    # logger.info(f"Mean: {np.shape(mean_pred)}\nVar: {np.shape(var_pred)}")

    # Extract Mean & Var for RI Data
    _sigmaR = np.array([var_pred[i][i] for i in range(n_RI)])
    _sigmaI = np.array([var_pred[i+n_RI][i+n_RI] for i in range(n_RI)])
    _covRI = np.array([var_pred[i][i+n_RI] for i in range(n_RI)])

    _meanR = mean_pred[:,0]
    _meanI = mean_pred[:,1]

    # Calculate Mean & Var for AP Data
    _amp_mean = np.abs(_meanR+1j*_meanI)
    _phz_mean = np.angle(_meanR+1j*_meanI)

    _amp_var = np.sqrt(((_meanR**2)*_sigmaR + (_meanI**2)*_sigmaI + 2*_meanR*_meanI*_covRI))/(_amp_mean)
    _phz_var = np.sqrt(((_meanI**2)*_sigmaR + (_meanR**2)*_sigmaI - 2*_meanR*_meanI*_covRI))/(_amp_mean**2)

    _phz_mean = np.rad2deg(_phz_mean)
    _phz_var = np.rad2deg(_phz_var)

    EIS_org.append([])
    EIS_GP.append([[_amp_mean,_amp_var],[_phz_mean, _phz_var]])

    logger.info(f"Freq: {i} - Done ")
    # logger.info(f"Freq: {i} - Done \n{np.shape(np.array([[_amp_mean,_amp_var],[_phz_mean, _phz_var]]))} \n{np.shape(_amp_mean)}")

EIS_GP = np.stack(EIS_GP, axis=1)
EIS_GP = np.transpose(EIS_GP, (0,2,1,3))
logger.info(np.shape(EIS_GP))





_poi_point = 50

plt.figure(figsize=(8,6))
plt.plot(x_eval, EIS_GP[0][0][_poi_point], 'b.', label='Mean Prediction')
plt.fill_between(x_eval, EIS_GP[0][0][_poi_point] - 2*np.sqrt(EIS_GP[0][1][_poi_point]), EIS_GP[0][0][_poi_point] + 2*np.sqrt(EIS_GP[0][1][_poi_point]), 
                 alpha=0.3, color='blue', label='95% CI')
plt.plot(x_train, np.abs(chData[:,1,freq_list[_poi_point]] + 1j*chData[:,1,freq_list[_poi_point]]), 'r.', label='Mean Train')
# plt.plot(x_eval, EIS_GP[1][0][0], 'b.', label='Mean Prediction')
# plt.fill_between(x_eval, EIS_GP[1][0][0] - 2*EIS_GP[1][1][0], EIS_GP[1][0][0] + 2*EIS_GP[1][1][0], 
#                  alpha=0.3, color='red', label='95% CI')
plt.xlabel('x')
plt.ylabel('y')
plt.title("Single-task GPR: Mean and Variance")
plt.legend()
plt.show()




c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages\gpytorch\models\exact_gp.py:296: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
2025-02-26 19:06:19.558 | INFO     | __main__:<module>:76 - Freq: 0 - Done 
c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages\gpytorch\models\exact_gp.py:296: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
2025-02-26 19:06:19.741 | INFO     | __main__:<module>:76 - Freq: 49 - Done 
c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages\gpytorch\models\exact_gp.py:296: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
2025-02-26 19:06:19.958 | INFO     | __main__:<module>:76 - Freq: 99 - Done 
c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages\gpytorch\models\exact_gp.py:296: GPInputWarning: The input matches the stored training data. Did you forget to

In [96]:
amp = EIS_GP[0]
phz = EIS_GP[1]
# logger.info(f"amp: {np.shape(amp)}, phz: {np.shape(phz)}")


fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')
init_elev = 21  # 仰角
init_azim = 55  # 方位角
ax1.view_init(elev=init_elev, azim=init_azim)
ax2.view_init(elev=init_elev, azim=init_azim)


y = np.array(x_eval).flatten()
x = np.log10(chData[0,0,freq_list]).flatten()
X, Y = np.meshgrid(x, y, indexing='ij')


ax1.plot_surface(X, Y, np.log10(amp[0]), cmap='viridis_r')
ax2.plot_surface(X, Y, phz[0], cmap='viridis')


# amp_varSpace = plotMeanVar(np.log10(amp[0]+2*amp[1]), np.log10(amp[0]-2*amp[1]), X, Y)
# phz_varSpace = plotMeanVar(phz[0]+2*phz[1],phz[0]-2*phz[1], X, Y)

# ax1.add_collection3d(amp_varSpace)
# ax2.add_collection3d(phz_varSpace)

ax1.set_zlim([2,8.5])
ax2.set_zlim([-120,30])


(-120.0, 30.0)